### Instrumented Principal Component Analysis

Let $t = 1,...,T$ index time and $i = 1,...,N$ index individuals. The IPCA model is:

\begin{equation}
\tag{1}
x_{i,t} = \beta_{i,t} f_t + \mu_{i,t},
\end{equation}

\begin{equation}
\tag{2}
\beta_{i,t} = c_{i,t} \Gamma + \eta_{i,t}.
\end{equation}

where $f_t$ is $K\times 1$ factors, $\Gamma$ is $L\times K$ matrix, number of covariates is $L$ in $c_{i,t}$. The matrix $\Gamma$ determines how each of the $L$ attributes maps into each of the $K$ factor loadings.

Plug $\beta_{i,t}$ in equation (2) into equation (1) and combine their two errors into a new compound error term $\epsilon_{i,t}$
\begin{equation}
\tag{3}
x_{i,t} = C_{i,t} \Gamma f_t + e_{i,t}, \quad e_{i,t} := \eta_{i,t} f_t + \mu_{i,t}.
\end{equation}



The optimization does not admit an analytical solution in general, but is speed- ily solved numerically by an alternating least squares (ALS) algorithm. 

### Data generating process

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
colors = sns.color_palette()
colors

[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7372549019607844, 0.7411764705882353, 0.13333333333333333),
 (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)]

In [88]:
K = 2 # number of latent factors
L = 10 # number of covariates
N = 30 # number of units
T = 50 # number of time periods

# generate fators f follow a VAR(1) process
coef_f = np.random.randn(K, K) # Coefficient matrix coef_f for the VAR(1) process
# initialize factors
F = np.zeros((T, K))
F[0] = np.random.randn(K)
for t in range(1, T):
    epsilon = np.random.multivariate_normal(np.zeros(K), np.eye(K))
    F[t] = coef_f@F[t-1] + epsilon

# generate covariates C for each unit i and time t, which follow a VAR(1) process
coef_c = np.random.randn(L, L) # Coefficient matrix coef_c for the VAR(1) process
# initialize covariates
C = np.zeros((N, T, L))
for i in range(N):
    C[i, 0] = np.random.randn(L)
    for t in range(1, T):
        epsilon = np.random.multivariate_normal(np.zeros(L), np.eye(L))
        C[i, t] = coef_c@C[i, t-1] + epsilon

# generate error terms
mu = np.random.randn(N, T)
eta = np.random.randn(N, T, K)

# define Gamma
Gamma = np.column_stack((np.arange(1, L + 1), np.arange(L, 0, -1)))

In [89]:
# compute X
X = np.zeros((N, T))
for i in range(N):
    for t in range(T):
        X[i, t] = C[i, t]@Gamma@F[t] + eta[i,t]@F[t] + mu[i, t]

In [113]:
# we consider two covariates and two factors
def generate_dataframe(N_co, N_tr, T0, T1, w):
    '''
    N_co: number of control units
    N_tr: number of treated units
    T0: number of pre-treatment periods
    T1: number of post-treatment periods
    w: similarity between treated and control units, 1 means identical
    '''
    # Constants
    N = N_co + N_tr
    T = T0 + T1
    ss = np.sqrt(3)

    # Random components
    epsilon, eta, f, xi = [np.random.normal(0, 1, size=s) for s in [(N, T), (2, N, T), (2, T), T]]
    lambda_co, alpha_co = [np.random.uniform(-ss, ss, size=s) for s in [(2, N_co), N_co]]
    lambda_tr, alpha_tr = [np.random.uniform(ss-2*w*ss, 3*ss-2*w*ss, size=s) for s in [(2, N_tr), N_tr]]

    # Factor loadings and unit fixed effects
    lambda_ = np.concatenate([lambda_co, lambda_tr], axis=1)
    alpha = np.concatenate([alpha_co, alpha_tr])

    # Treatment effects and assignment
    tr = np.concatenate([np.zeros(N_co), np.ones(N_tr)])
    delta = np.concatenate([np.zeros(T0), np.arange(1, T1+1)+ np.random.normal(0, 1, T1)]) 

    # Covariates and outcome
    # here the author didn't mention that he scaled some components in the paper but he did in his code
    x1, x2 = [1 + 0.5*(lambda_.T @ f) + 0.25*(lambda_[0].reshape(-1, 1) + lambda_[1].reshape(-1, 1)) + 0.25*(f[0] + f[1]) + eta[i] for i in range(2)]
    y = tr.reshape(-1,1)@delta.reshape(1, -1) + x1*1 + x2*3 + lambda_.T@f + alpha.reshape(-1,1) + xi + epsilon + 5

    # Construct DataFrame
    df = pd.DataFrame({
        'id': np.repeat(np.arange(101, N + 101), T),
        'year': np.tile(np.arange(1, T + 1), N),
        'y': y.flatten(),
        'd': np.repeat(tr, T),
        'x1': x1.flatten(),
        'x2': x2.flatten(),
        'eff': np.tile(delta, N),
        'error': epsilon.flatten(),
        'mu': 5,
        'alpha': np.repeat(alpha, T),
        'xi': np.tile(xi, N),
        'f1': np.tile(f[0], N),
        'l1': np.repeat(lambda_[0], T),
        'f2': np.tile(f[1], N),
        'l2': np.repeat(lambda_[1], T)
    })

    return df

In [121]:
N_co=25
N_tr=5
T0=40
T1=10
w=0.8
datt = generate_dataframe(N_co, N_tr, T0, T1, w)

In [124]:
X = datt.pivot(index='id', columns='year', values='y')
C = datt.pivot(index='id', columns='year', values=['x1', 'x2'])
C = C.values.reshape(N, T, 2)

### Estimation
---
IPCA is estimated as a least squares problem that minimizes the sample sum of squared errors (SSE) over parameters $\Gamma,  f_t$:

$$
\tag{4}
\min_{\Gamma, \{f_t\}} \sum_{i,t} (x_{i,t} - c_{i,t} \Gamma f_t)^2 .
$$

We use an Alternating Least Squares (ALS) method for the numerical solution of the optimization because, unlike PCA, the IPCA optimization problem does not have a solution through an eigen-decomposition.

Given any $\Gamma$, factors $f_t$ are $t$-separable and solved with cross-sectional OLS for each $t$:
$$
\tag{5}
\hat{f}_t(\Gamma) := \arg\min_{f_t} \sum_{i} (x_{i,t} - c_{i,t} \Gamma f_t)^2 = (\Gamma^T C_t^T C_t \Gamma)^{-1} \Gamma^T C_t^T x_t.
$$


Symmetrically, given $f_t$, the optimizing $\Gamma$ (vectorized as $\gamma$) is solved with pooled panel OLS of $x_{i,t}$ onto $LK$ regressors, $c_{i,t} \otimes f_t^T$

$$
\tag{6}
\hat{\Gamma}(f_t):=\arg\min_{\gamma} \sum_{i,t} (x_{i,t} - c_{i,t} \Gamma f_t)^2 = \left( \sum_{i,t} (c_{i,t}^T \otimes f_t) (c_{i,t} \otimes f_t) \right)^{-1} \left( \sum_{i,t} (c_{i,t}^T \otimes f_t) x_{i,t} \right).
$$

$$
\tag{6}
\hat{\Gamma}(f_t) = \left( \sum_{i,t} (c_{i,t}^T \otimes f_t) (c_{i,t} \otimes f_t) \right)^{-1} \left( \sum_{i,t} (c_{i,t}^T \otimes f_t) x_{i,t} \right).
$$

I have X matrix (N,T), C matrix (N,T,L), I have Ft matrix (K,T), how to write a this function in python to compute Gamma?

In [81]:
def als_estimator(X, C, K, max_iter=100):
    """
    Estimate the factors F and the matrix Gamma using the alternating least squares (ALS) algorithm.
    :param X: N x T matrix of observed data
    :param C: N x T x L matrix of covariates
    :param K: number of latent factors
    :return: F: K x T matrix of factors
             Gamma: L x K matrix of coefficients
    """
    # initialize beta and Ft via SVD
    U, S, Vt = np.linalg.svd(X)
    Ft = np.diag(S[:K])@Vt[:K]

    for i in range(max_iter):

        # update Gamma
        # Sum over all i and t
        for t in range(T):
            for i in range(N):
                # Compute the Kronecker product
                ci_t = C[i, t, :].reshape(L, 1)  # Reshape to column vector
                ft = Ft[:, t].reshape(K, 1)      # Reshape to column vector
                kron_product = np.kron(ci_t, ft)

        # Initialize the sums
        sum_kron_product = np.zeros((L * K, L * K))
        sum_kron_x = np.zeros((L * K, 1))

        # Sum over all i and t
        for t in range(T):
            for i in range(N):
                # Compute the Kronecker product
                ci_t = C[i, t, :].reshape(L, 1)  # Reshape to column vector
                ft = Ft[:, t].reshape(K, 1)      # Reshape to column vector
                kron_product = np.kron(ci_t, ft)

                # Update the sums
                sum_kron_product += kron_product @ kron_product.T
                sum_kron_x += kron_product * X[i, t]  # Element-wise multiplication

        # Compute Gamma using the pseudo-inverse for numerical stability
        Gamma_vec = np.linalg.pinv(sum_kron_product) @ sum_kron_x
        Gamma = Gamma_vec.reshape(L, K)

        # update Factor
        Ft = (np.linalg.pinv(Gamma.T @ C[:, t, :].T @ C[:, t, :] @ Gamma) @ Gamma.T @ C[:, t, :].T @ X)
    return Gamma, Ft

In [98]:
g, f = als_estimator(X=X, C=C, K=2, max_iter=1000)

In [106]:
# initialize beta and Ft via SVD
U, S, Vt = np.linalg.svd(X)
Ft = np.diag(S[:K])@Vt[:K]

# update Gamma
# Sum over all i and t
for t in range(T):
    for i in range(N):
        # Compute the Kronecker product
        ci_t = C[i, t, :].reshape(L, 1)  # Reshape to column vector
        ft = Ft[:, t].reshape(K, 1)      # Reshape to column vector
        kron_product = np.kron(ci_t, ft)
# Initialize the sums
sum_kron_product = np.zeros((L * K, L * K))
sum_kron_x = np.zeros((L * K, 1))
# Sum over all i and t
for t in range(T):
    for i in range(N):
        # Compute the Kronecker product
        ci_t = C[i, t, :].reshape(L, 1)  # Reshape to column vector
        ft = Ft[:, t].reshape(K, 1)      # Reshape to column vector
        kron_product = np.kron(ci_t, ft)
        # Update the sums
        sum_kron_product += kron_product @ kron_product.T
        sum_kron_x += kron_product * X[i, t]  # Element-wise multiplication
# Compute Gamma using the pseudo-inverse for numerical stability
Gamma_vec = np.linalg.pinv(sum_kron_product) @ sum_kron_x
Gamma = Gamma_vec.reshape(L, K)

In [110]:
C

array([[[-1.89360510e+00,  5.32520760e-01,  1.66034719e-01, ...,
         -1.18032801e+00, -3.64878341e-01,  8.29102513e-01],
        [-5.67530428e+00,  9.66556740e-01, -1.11266720e-01, ...,
          5.43681542e+00, -3.95566205e-01, -1.40911770e+00],
        [-1.16543085e+01,  1.83762028e+00,  1.47094745e+01, ...,
         -3.62158743e+00, -3.83008874e-01, -1.43198970e+01],
        ...,
        [-9.36624661e+22,  7.25361001e+21,  8.00767754e+22, ...,
         -1.19711662e+21,  3.49848009e+22,  1.83230303e+22],
        [-4.68520566e+23, -9.59294885e+22,  2.86173022e+23, ...,
         -1.77972604e+22, -3.38162543e+22, -1.22409599e+23],
        [-1.59864770e+24, -5.99079204e+23,  6.86806243e+23, ...,
          7.89181162e+22, -5.27996651e+23, -7.97011480e+23]],

       [[-4.79390067e+00,  3.45575629e-01, -1.39972111e+00, ...,
         -3.50426659e-01, -9.24336426e-01,  2.15591897e-01],
        [-3.47270153e+00, -1.22997227e-02, -2.53573780e+00, ...,
          9.72532126e+00, -4.27751519e

To solve for $f_t$ analytically in the given optimization problem

$$
\min_{\Gamma, \{f_t\}} \sum_{i,t} (x_{i,t} - c_{i,t} \Gamma f_t)^2
$$

given that $x_{i,t}, c_{i,t}, \Gamma$ are known, we can approach the problem as follows. This optimization problem aims to minimize the squared difference between observed values $x_{i,t}$ and the model's prediction $c_{i,t} \Gamma f_t$.

For a given time $t$, the problem reduces to

$$
\min_{f_t} \sum_{i} (x_{i,t} - c_{i,t} \Gamma f_t)^2
$$

To solve for $f_t$, we differentiate this expression with respect to $f_t$ and set the derivative equal to zero to find the minimum.

Let's denote $X_t = [x_{1,t}, x_{2,t}, ..., x_{N,t}]^T$ as the vector of observed values at time $t$, $C_t = [c_{1,t}, c_{2,t}, ..., c_{N,t}]^T$ as the vector of coefficients at time $t$, and $\Gamma$ as the matrix that is pre-specified. The goal is to find $f_t$.

The objective function can be rewritten in matrix form as

$$
\min_{f_t} \|X_t - C_t \Gamma f_t\|^2
$$

Taking the derivative of this expression with respect to $f_t$ and setting it to zero gives

$$
-2C_t^T \Gamma^T (X_t - C_t \Gamma f_t) = 0
$$

Solving for $f_t$ gives

$$
C_t^T \Gamma^T C_t \Gamma f_t = C_t^T \Gamma^T X_t
$$

Assuming $C_t^T \Gamma^T C_t \Gamma$ is invertible, the solution for $f_t$ is

$$
f_t = (C_t^T \Gamma^T C_t \Gamma)^{-1} C_t^T \Gamma^T X_t
$$

This gives the analytical solution for $f_t$ given $X_t$, $C_t$, and $\Gamma$. Note that the invertibility of $C_t^T \Gamma^T C_t \Gamma$ is a key assumption here. If this matrix is not invertible, alternative methods such as regularization might be necessary to find a solution.